##### The following code analyzes the viable_patches_json file. The points of analysis are described below. The primary tool for this analysis is pydriller.

1. Total size of the cloned repos
2. Total number of vulnerability inducing commits (vuln commits) found & not found
3. Average number of months between vuln commit and patch commit (or fix)
4. Average number of commits between the vuln commit & patch commit (or fix)
5. Average number of vuln commits fixed by patch commit (or fix)
6. Percentage of vulns where the vuln commit and fix were made by the same person


##### Sources
- @inbook{PyDriller,
    title = "PyDriller: Python Framework for Mining Software Repositories",
    abstract = "Software repositories contain historical and valuable information about the overall development of software systems. Mining software repositories (MSR) is nowadays considered one of the most interesting growing fields within software engineering. MSR focuses on extracting and analyzing data available in software repositories to uncover interesting, useful, and actionable information about the system. Even though MSR plays an important role in software engineering research, few tools have been created and made public to support developers in extracting information from Git repository. In this paper, we present PyDriller, a Python Framework that eases the process of mining Git. We compare our tool against the state-of-the-art Python Framework GitPython, demonstrating that PyDriller can achieve the same results with, on average, 50% less LOC and significantly lower complexity.URL: https://github.com/ishepard/pydrillerMaterials: https://doi.org/10.5281/zenodo.1327363Pre-print: https://doi.org/10.5281/zenodo.1327411",
    author = "Spadini, Davide and Aniche, Maurício and Bacchelli, Alberto",
    year = "2018",
    doi = "10.1145/3236024.3264598",
    booktitle = "The 26th ACM Joint European Software Engineering Conference and Symposium on the Foundations of Software Engineering (ESEC/FSE)",
    }

##### Author @Trust-Worthy



##### Reading in the results from the patch_vuln_match.json file and processing objects according to JSONL standard

In [29]:
import pandas as pd
import jsonlines    

json_path:str = "../../production_ready/patch_vuln_match.jsonl"

data: list[object] = []

with jsonlines.open(json_path) as reader:

    data = [entry for entry in reader]

# Convert the list of dictionaries into a pandas DataFrame
patch_vuln_df = pd.DataFrame(data)


# Define a function to extract the file paths and commits
def extract_vuln_files_commits(vuln_commits):
    if vuln_commits:
        files = list(vuln_commits.keys())
        commits = [commit for commits in vuln_commits.values() for commit in commits]
        return pd.Series([files, commits])
    else:
        return pd.Series([[], []])  # Empty lists if no vuln_commits

# # Apply the function to create new columns
# patch_vuln_df[['vuln_files', 'vuln_commits']] = patch_vuln_df['vuln_commits'].apply(extract_vuln_files_commits)



# print(patch_vuln_df.head())


##### This is where the fun begins.... (iykyk)

In [30]:
def extract_file_paths(vuln_commits):
    try:
        if isinstance(vuln_commits, dict):
            return list(vuln_commits.keys())
        return []
    except Exception as e:
        print(f"Error extracting file paths: {e}")
        return []
def extract_commit_hashes(vuln_commits):
    try:
        if isinstance(vuln_commits, dict):
            return list({commit for commits in vuln_commits.values() if isinstance(commits, list) for commit in commits})
        return []
    except Exception as e:
        print(f"Error extracting commit hashes: {e}")
        return []


In [31]:
# Apply functions to create new columns
patch_vuln_df["vuln_files"] = patch_vuln_df["vuln_commits"].apply(extract_file_paths)
patch_vuln_df["vuln_hashes"] = patch_vuln_df["vuln_commits"].apply(extract_commit_hashes)



print(patch_vuln_df.head())
print(patch_vuln_df.tail())

           cve_id                     repo  \
0   CVE-1999-0199             bminor/glibc   
1   CVE-1999-0731         KDE/kde1-kdebase   
2   CVE-2002-2443                krb5/krb5   
3  CVE-2005-10002  wp-plugins/secure-files   
4  CVE-2005-10003      mikexstudios/xcomic   

                               patch_commit  \
0  2864e767053317538feafa815046fff89e5a16be   
1  04906bd5de2f220bf100b605dad37b4a1d9a91a6   
2  cf1a0c411b2668c57c41e9c4efd15ba17b6b322c   
3  cab025e5fc2bcdad8032d833ebc38e6bd2a13c92   
4  6ed8e3cc336e29f09c7e791863d0559939da98bf   

                                        vuln_commits  \
0  {'elf/dl-load.c': ['dc5efe83c0252ad45337ab98ef...   
1                                                 {}   
2  {'src/kadmin/server/schpw.c': ['e88f857c3680ea...   
3  {'secure-files.php': ['b1afc063fd49cfb875e1c6f...   
4                                                 {}   

                                          vuln_files  \
0  [elf/dl-load.c, manual/search.texi, misc/sys

In [32]:
# Drop the original vuln_commits column if not needed
patch_vuln_df.drop(columns=["vuln_commits"], inplace=True)

print(patch_vuln_df.head())
print(patch_vuln_df.tail())

           cve_id                     repo  \
0   CVE-1999-0199             bminor/glibc   
1   CVE-1999-0731         KDE/kde1-kdebase   
2   CVE-2002-2443                krb5/krb5   
3  CVE-2005-10002  wp-plugins/secure-files   
4  CVE-2005-10003      mikexstudios/xcomic   

                               patch_commit  \
0  2864e767053317538feafa815046fff89e5a16be   
1  04906bd5de2f220bf100b605dad37b4a1d9a91a6   
2  cf1a0c411b2668c57c41e9c4efd15ba17b6b322c   
3  cab025e5fc2bcdad8032d833ebc38e6bd2a13c92   
4  6ed8e3cc336e29f09c7e791863d0559939da98bf   

                                          vuln_files  \
0  [elf/dl-load.c, manual/search.texi, misc/syslo...   
1                                                 []   
2                        [src/kadmin/server/schpw.c]   
3                                 [secure-files.php]   
4                                                 []   

                                         vuln_hashes  
0  [569c558c880779d33c6642662d1aa57dff697244, 26

In [33]:
"""
Global variables
"""
NVD_ALL_REPOS = "/shared/rc/sfs/nvd-all-repos"

MATCH_FILES:str = "../../production_ready/patch_vuln_match.jsonl"

### Point 1
SIZE_OF_ALL_CLONED_REPOS: float = 0 ### size in MB

### Point 2
TOTAL_VULNS_COMMITS: int = 0 ### Another way to say this is total patch vuln pairs
TOTAL_PATCH_COMMITS_W_VULN_COMMIT: int = 0


### Point 6
### I can get the the number of patches without vulns / not found by doing total entires - total vulns
BY_SAME_PERSON: int = 0 ### Num of vulns made by the same person
PERCENTAGE_OF_VULN_N_PATCH_BY_SAME_PERSON: float = 0.0


### Point 3
TOTAL_NUM_MONTHS_BETWEEN: int = 0
AVERAGE_NUM_MONTHS_BETWEEN_VULN_N_PATCH: float = 0.0

### Point 4
TOTAL_NUM_COMMITS_BETWEEN: int = 0
AVERAGE_NUM_COMMITS_BETWEEN_VULN_N_PATCH: float = 0.0


In [43]:
import os
import logging
import glob
# Configure logging
logging.basicConfig(
filename="test_jupyter_1.log",
level=logging.WARNING,
format="%(asctime)s - %(levelname)s - %(message)s",
)

# Calculate repo size
def get_directory_size(path: str) -> float:
    size: float = 0
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            size += os.path.getsize(fp)
    logging.info(f"got the size for {path} repo")
    return size

def safe_extract_vuln_files_commits(vuln_commits):
    """Wrapper function for error handling and logging."""
    try:
        return extract_vuln_files_commits(vuln_commits)
    except Exception as e:
        logging.error(f"Error processing vuln_commits: {vuln_commits} - {e}", exc_info=True)
        return pd.Series([[], []])  # Return empty lists in case of failure
def find_repo_path(owner_repo: str) -> str | None:
    """Finds the path of a repository inside NVD_ALL_REPOS.

    Args:
        owner_repo (str): The repository in 'owner/repo' format.

    Returns:
        str | None: The path to the repository if found, otherwise None.
    """
    
    matching_repos:list = glob.glob(os.path.join(NVD_ALL_REPOS, f"*{owner_repo}*"))
    return matching_repos



In [17]:
from typing import Optional
from pydriller import Repository
from datetime import datetime
from dateutil.relativedelta import relativedelta 
'''
POINT 1

'''

def calculate_all_repo_sizes(patch_vuln_df: pd.DataFrame) -> float:
    """
    Calculates the total disk size of all unique repositories in the given DataFrame.

    This function iterates through the "repo" column, determines their local paths,
    and accumulates their sizes (in MB), ensuring each repo is counted only once.

    Args:
        patch_vuln_df (pd.DataFrame): DataFrame containing repo names.

    Returns:
        float: Total repository size in MB.
    """
    unique_repo_paths: set[str] = set()
    total_size: float = 0.0  # Initialize total size inside the function

    for repo in patch_vuln_df["repo"]:
        try:
            repo_path = find_repo_path(repo)
            if not repo_path:
                logging.warning(f"Repository path not found for {repo}, skipping.")
                continue

            if repo_path in unique_repo_paths:
                continue  # Skip if already counted

            unique_repo_paths.add(repo_path)
            repo_size: float = get_directory_size(repo_path) / (1024 * 1024)  # Convert to MB
            total_size += repo_size

            logging.info(f"Added {repo_path} ({repo_size:.2f} MB). Total: {total_size:.2f} MB")

        except Exception as e:
            logging.error(f"Error processing {repo}: {e}")
            continue  # Move to the next repo

    return total_size


In [35]:
'''
POINT 2
'''
def calculate_total_num_vuln_hashes(patch_vuln_df: pd.DataFrame) -> int:
    return patch_vuln_df["vuln_hashes"].explode().count()

def calculate_patch_vuln_matches(patch_vuln_df: pd.DataFrame) -> int:
    
    # Query for empty vuln_files and vuln_hashes
    empty_patch_vuln_matches_count = patch_vuln_df[
        #patch_vuln_df["vuln_files"].apply(lambda x: len(x) == 0) #& 
        patch_vuln_df["vuln_hashes"].apply(lambda x: len(x) > 0)
    ].shape[0]
    return empty_patch_vuln_matches_count

TOTAL_VULNS_COMMITS = calculate_total_num_vuln_hashes(patch_vuln_df)
TOTAL_PATCH_COMMITS_W_VULN_COMMIT = calculate_patch_vuln_matches(patch_vuln_df)
TOTAL_DF_ENTRIES = patch_vuln_df.shape[0]
print("Total entries in patch vuln dataframe:" + str(TOTAL_DF_ENTRIES))
print("Total vulnerable commits in patch vuln dataframe:" + str(TOTAL_VULNS_COMMITS)) ### really total vuln patches
print("Total patch commits with at least one vulnerable commit:" + str(TOTAL_PATCH_COMMITS_W_VULN_COMMIT))

Total entries in patch vuln dataframe:13383
Total vulnerable commits in patch vuln dataframe:47979
Total patch commits with at least one vulnerable commit:10271


In [45]:
'''
Point 3
'''
non_empty_vuln_hashes_df = patch_vuln_df[patch_vuln_df["vuln_hashes"].apply(lambda x: len(x) > 0)].copy()

print(non_empty_vuln_hashes_df.head())
def calculate_total_num_months_between_patch_and_vulns(non_empty_vuln_hashes: pd.DataFrame) -> int:
    total = 0

    for repo, patch_commit_hash, vuln_hashes in zip(non_empty_vuln_hashes["repo"], non_empty_vuln_hashes["patch_commit"], non_empty_vuln_hashes["vuln_hashes"]):
        
        # repo_path = find_repo_path(repo)
        repo_url = "https://github.com/" + repo
        commits_to_analyze = [patch_commit_hash] + vuln_hashes  # Add patch commit + all vuln commits

        if not patch_commit_hash or not vuln_hashes:
            logging.warning(f"Skipping {repo} due to missing patch or vulnerability hashes.")
            continue
        
        try:    
            REPOSITORY = Repository(repo_url, only_commits=commits_to_analyze, order='reverse')
        except Exception as e:
            logging.error(f"Failed to initialize repository for {repo}: {e}")
            continue

        patch_commit_date = None
        total_diff_in_months = 0

        # Process commits one by one
        for commit in REPOSITORY.traverse_commits():
            if commit.hash == patch_commit_hash:
                patch_commit_date = commit.author_date  # Store patch commit date
                print(f"Commit hash {commit.hash} == patch commit hash {patch_commit_hash}")
                continue

            if patch_commit_date:
                total_diff_in_months += abs((patch_commit_date - commit.author_date).days) / 30.44  # Convert days to months
        print(f"Total diff in months for {repo} is {total_diff_in_months}")
        total += total_diff_in_months

    return total




           cve_id                              repo  \
0   CVE-1999-0199                      bminor/glibc   
2   CVE-2002-2443                         krb5/krb5   
3  CVE-2005-10002           wp-plugins/secure-files   
5  CVE-2006-10001  wp-plugins/subscribe-to-comments   
6   CVE-2006-3635                    torvalds/linux   

                               patch_commit  \
0  2864e767053317538feafa815046fff89e5a16be   
2  cf1a0c411b2668c57c41e9c4efd15ba17b6b322c   
3  cab025e5fc2bcdad8032d833ebc38e6bd2a13c92   
5  9683bdf462fcac2f32b33be98f0b96497fbd1bb6   
6  4dcc29e1574d88f4465ba865ed82800032f76418   

                                          vuln_files  \
0  [elf/dl-load.c, manual/search.texi, misc/syslo...   
2                        [src/kadmin/server/schpw.c]   
3                                 [secure-files.php]   
5                        [subscribe-to-comments.php]   
6                           [arch/ia64/kernel/ivt.S]   

                                         vuln_has

In [46]:
total = calculate_total_num_months_between_patch_and_vulns(non_empty_vuln_hashes_df)
print(f"Total is: {total}")

Commit hash 2864e767053317538feafa815046fff89e5a16be == patch commit hash 2864e767053317538feafa815046fff89e5a16be
Total diff in months for bminor/glibc is 640.7030223390275
Commit hash cf1a0c411b2668c57c41e9c4efd15ba17b6b322c == patch commit hash cf1a0c411b2668c57c41e9c4efd15ba17b6b322c
Total diff in months for krb5/krb5 is 208.01576872536137
Commit hash cab025e5fc2bcdad8032d833ebc38e6bd2a13c92 == patch commit hash cab025e5fc2bcdad8032d833ebc38e6bd2a13c92
Total diff in months for wp-plugins/secure-files is 0.19710906701708278
Commit hash 9683bdf462fcac2f32b33be98f0b96497fbd1bb6 == patch commit hash 9683bdf462fcac2f32b33be98f0b96497fbd1bb6
Total diff in months for wp-plugins/subscribe-to-comments is 19.250985545335084
Commit hash 4dcc29e1574d88f4465ba865ed82800032f76418 == patch commit hash 4dcc29e1574d88f4465ba865ed82800032f76418
Total diff in months for torvalds/linux is 37.319316688567675
Commit hash 3e8f0717f133907bb6a09cb674b05c5f273da3e1 == patch commit hash 3e8f0717f133907bb6a09

KeyboardInterrupt: 

In [ ]:

'''
POINT 4
'''


def calculate_total_num_commits_between_patch_and_vulns(non_empty_vuln_hashes: pd.DataFrame) -> int:
    
    total_commits_between: int = 0
    for repo,patch_commit_hash,vuln_hashes in zip(non_empty_vuln_hashes["repo"],non_empty_vuln_hashes["patch_commit"],non_empty_vuln_hashes["vuln_hashes"]):
        
        repo_path: str = find_repo_path(repo)
        
        for vuln_hash in vuln_hashes:
            try:
                REPOSITORY: Repository = Repository(repo_path, from_commit=vuln_hash, to_commit=patch_commit_hash, order='reverse')
                num_commits_between = sum(1 for _ in REPOSITORY.traverse_commits())
                total_commits_between += num_commits_between
            except Exception as e:
                logging.error(f"Failed to initialize repository for {repo}: {e}")
            continue

    return total_commits_between
            
        
       

In [ ]:
'''
POINT 5
'''

def calculate_average_num_vuln_commits_fixed_by_patch_commit(
        total_vulns,patch_vuln_matches) -> float:
    return total_vulns / patch_vuln_matches


In [ ]:
'''
POINT 6
'''

def calculate_num_vulns_made_and_fixed_by_same_person(non_empty_vuln_hashes: pd.DataFrame)->int:
    total = 0

    for repo, patch_commit_hash, vuln_hashes in zip(non_empty_vuln_hashes["repo"], non_empty_vuln_hashes["patch_commit"], non_empty_vuln_hashes["vuln_hashes"]):
        
        repo_path = find_repo_path(repo)
        commits_to_analyze = [patch_commit_hash] + vuln_hashes  # Add patch commit + all vuln commits

        if not patch_commit_hash or not vuln_hashes:
            logging.warning(f"Skipping {repo} due to missing patch or vulnerability hashes.")
            continue
        
        try:    
            REPOSITORY = Repository(str(repo_path), only_commits=commits_to_analyze, order='reverse')
        except Exception as e:
            logging.error(f"Failed to initialize repository for {repo}: {e}")
            continue

        commit = next(Repository(repo_path, single=patch_commit_hash).traverse_commits())
        patch_commit_author = commit.author.email

        occurances: int = 0

        # Process commits one by one
        for commit in REPOSITORY.traverse_commits():
            if commit.author.email == patch_commit_author:
                occurances += 1


        total += occurances

    return total
    